In [57]:
from diffusers import StableDiffusionXLPipeline
import torch
from accelerate import infer_auto_device_map
import random
import torch
from PIL import Image

print("CUDA available:", torch.cuda.is_available())
print("Number of GPUs:", torch.cuda.device_count())
print("Current device:", torch.cuda.current_device())
print("Device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU detected")

CUDA available: True
Number of GPUs: 1
Current device: 0
Device name: NVIDIA GeForce RTX 4080 SUPER


In [58]:
# Carica il modello dalla piattaforma Hugging Face
model_id = "stabilityai/stable-diffusion-xl-base-1.0"

# Configura la pipeline
pipeline = StableDiffusionXLPipeline.from_pretrained(
    model_id, 
    torch_dtype=torch.float16, 
    use_safetensors=True,  # Per performance migliori
).to("cuda")

Loading pipeline components...: 100%|██████████| 7/7 [00:02<00:00,  2.75it/s]


In [63]:
prompt = "Mc donalds's Nugget on a snowboard with a cowboy hat, in front of a snowy mountainous landscape with clear weather."
negative = "blurry, low quality, pixelated, out of focus, overexposed, underexposed, poorly detailed, text artifacts, watermark, grainy textures, flat colors, monochrome, color bleeding, low resolution, bad composition."

In [64]:
num_inference_steps=50
guidance_scale=7.5

height = 1024  # Altezza in pixel
width = 1024  # Larghezza in pixel

# random seed
seed = random.randint(1,1000)
print("Using Seed: ",seed)
generator = torch.Generator(device="cpu").manual_seed(seed)

# Generazione dell'immagine

### strenght  
image = pipeline(
    prompt=prompt,
    negative_prompt=negative,                   # Prompt negativo
    height=height,
    width=width,
    num_inference_steps=num_inference_steps,    # Passi di inferenza (maggiore = più dettagli)
    guidance_scale= guidance_scale,              # Peso per il prompt (maggiore = risultati più aderenti)
    generator=generator
).images[0]

# Salva l'immagine generata
image.save(f"imgs/{str(seed)}.png")

Using Seed:  307


100%|██████████| 50/50 [00:07<00:00,  6.25it/s]


# IMG TO IMG

In [78]:
import torch
from diffusers import StableDiffusionImg2ImgPipeline
from PIL import Image
import random
import os

# Set parameters
model_id = "stabilityai/stable-diffusion-xl-base-1.0"

In [ ]:
input_image_path = "/media/SSD_mmlab/martina.dangelo/fake-image-detectors/SD_script/input_imgs/nug.jpg"  # Path to your input image
output_folder = "imgs"
os.makedirs(output_folder, exist_ok=True)  # Ensure output directory exists

prompt = "A futuristic cityscape illuminated by neon lights"
negative = "blurry, low quality, pixelated, cartoonish, text artifacts"
num_inference_steps = 50
guidance_scale = 7.5
strength = 0.75  # Degree of transformation (0.0 = no change, 1.0 = full change)
height = 1024  # Image height
width = 1024   # Image width

# Load the pipeline
pipeline = StableDiffusionImg2ImgPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    use_safetensors=True,
).to("cuda")

# Load and preprocess the input image
if not os.path.exists(input_image_path):
    raise FileNotFoundError(f"Input image '{input_image_path}' not found.")
    
init_image = Image.open(input_image_path).convert("RGB")
init_image = init_image.resize((width, height))  # Resize to model dimensions

# Validate prompt
if not prompt:
    raise ValueError("Prompt cannot be empty.")

# Set random seed for reproducibility
seed = random.randint(1, 1000)
print("Using Seed: ", seed)
generator = torch.Generator(device="cuda").manual_seed(seed)  # Use "cuda" for the generator if running on GPU

# Generate the image
image = pipeline(
    prompt=prompt,
    negative_prompt=negative,
    image=init_image,                          # Input image
    strength=strength,                         # Degree of transformation
    num_inference_steps=num_inference_steps,
    guidance_scale=guidance_scale,
    generator=generator,
).images[0]

# Save the generated image
output_path = os.path.join(output_folder, f"{str(seed)}.png")
image.save(output_path)

print(f"Generated image saved at: {output_path}")


Loading pipeline components...: 100%|██████████| 5/5 [00:01<00:00,  2.65it/s]
You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion_img2img.StableDiffusionImg2ImgPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


Using Seed:  431


  0%|          | 0/37 [00:00<?, ?it/s]


TypeError: argument of type 'NoneType' is not iterable

# img to img

In [79]:
# import torch
# from diffusers import StableDiffusionImg2ImgPipeline
# from PIL import Image
# import random
# import os

# # Parameters
# model_id = "runwayml/stable-diffusion-v1-5"  # Compatible model for img2img
# output_folder = "imgs"
# input_image_path = "/media/SSD_mmlab/martina.dangelo/fake-image-detectors/SD_script/input_imgs/nug.jpg"  # Path to your input image

# os.makedirs(output_folder, exist_ok=True)  # Ensure output directory exists

# prompt = (
#     "A highly detailed and realistic depiction of a golden, crispy chicken nugget on a snowboard, wearing a classic cowboy hat. "
#     "The nugget is situated in a snowy mountainous landscape under clear blue skies. "
#     "The composition is photorealistic and humorous, with the nugget blending seamlessly into the environment."
# )
# negative = (
#     "blurry, low quality, pixelated, out of focus, poorly detailed, cartoonish, "
#     "text artifacts, watermarks, deformed features, oversaturated colors, "
#     "grainy, flat shading, exaggerated proportions, unrealistic textures, bad composition, noise."
# )


# num_inference_steps = 50
# guidance_scale = 7.5
# strength = 0.75  # Degree of transformation (0.0 = no change, 1.0 = full transformation)
# height = 512  # Image height (adjust as needed)
# width = 512   # Image width (adjust as needed)

# # Load the img2img pipeline
# pipeline = StableDiffusionImg2ImgPipeline.from_pretrained(
#     model_id,
#     torch_dtype=torch.float16,
# ).to("cuda")

# # Load and preprocess the input image
# if not os.path.exists(input_image_path):
#     raise FileNotFoundError(f"Input image '{input_image_path}' not found.")

# init_image = Image.open(input_image_path).convert("RGB")
# init_image = init_image.resize((width, height))  # Resize to model dimensions

# # Set random seed for reproducibility
# seed = random.randint(1, 1000)
# print("Using Seed:", seed)
# generator = torch.Generator(device="cuda").manual_seed(seed)

# # Generate the image
# image = pipeline(
#     prompt=prompt,
#     negative_prompt=negative,
#     image=init_image,                          # Input image
#     strength=strength,                         # Degree of transformation
#     num_inference_steps=num_inference_steps,
#     guidance_scale=guidance_scale,
#     generator=generator,
# ).images[0]

# # Save the generated image
# output_path = os.path.join(output_folder, f"{str(seed)}.png")
# image.save(output_path)

# print(f"Generated image saved at: {output_path}")


# Cropper

In [28]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import random
import os

PATCH_SIZE = 1024
DEBUG = False
WRITE = True
FOLDER_PATH = "imgs/"
OUTPUT_FOLDER_PATH = "cropped_imgs/"
FILE_FORMAT = (".png", ".jpg")


# Recupera lista img da FOLDER_PATH
images = [os.path.join(FOLDER_PATH, file) for file in os.listdir(FOLDER_PATH) if file.lower().endswith(FILE_FORMAT)]

print("Total images: ",len(images))
os.makedirs(OUTPUT_FOLDER_PATH, exist_ok=True)  # Crea la directory se non esiste

PROCESSED_IMG = len(images) 
PATCH_FOR_IMG = 4

# Process x images
random_image_list = random.sample(images, PROCESSED_IMG)

counter_tot = PROCESSED_IMG * PATCH_FOR_IMG
current_counter = 1
for file_name in random_image_list:
    img = cv2.imread(file_name)

    max_y = img.shape[0]-PATCH_SIZE
    max_x = img.shape[1]-PATCH_SIZE

    file_name_no_ext, ext = os.path.splitext(os.path.basename(file_name))

    # Crop x patches
    for patch_number in range(0, PATCH_FOR_IMG):
        top_left = (random.randint(0, max_x), random.randint(0, max_y))
        bot_right = (top_left[0]+PATCH_SIZE, top_left[1]+PATCH_SIZE)        

        cropped_image = img[top_left[1]:top_left[1]+PATCH_SIZE, top_left[0]:top_left[0]+1024]

        if(DEBUG):
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            cropped_image_rgb = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2RGB)

            cv2.rectangle(img_rgb, top_left, bot_right, (255, 0, 0), 20)  # Drawing in RGB

            plt.figure(figsize=(10, 5))
            plt.subplot(1, 2, 1)
            plt.imshow(img_rgb)
            plt.title("Original Image")
            plt.axis("off")

            plt.subplot(1, 2, 2)
            plt.imshow(cropped_image_rgb)
            plt.title("Cropped Patch")
            plt.axis("off")
            plt.show()

        save_name = OUTPUT_FOLDER_PATH + "/" + file_name_no_ext + "-" + str(patch_number+1) + ".png"

        if WRITE:
            cv2.imwrite(str(save_name), cropped_image)

        print(str(current_counter) + "/" + str(counter_tot))
        current_counter += 1

Total images:  4
1/16
2/16
3/16
4/16
5/16
6/16
7/16
8/16
9/16
10/16
11/16
12/16
13/16
14/16
15/16
16/16
